In [1]:
# Cell 1: Imports & Load Data
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

BASE_DIR = Path().resolve().parent
PROC_DIR = BASE_DIR / "data_processed"

# Load processed tables
dim_channel = pd.read_csv(PROC_DIR / "dim_channel.csv")
fact_marketing = pd.read_csv(PROC_DIR / "fact_marketing.csv")
fact_revenue = pd.read_csv(PROC_DIR / "fact_revenue.csv")

print("Data loaded:")
print(f"Channels: {len(dim_channel)}")
print(f"Marketing records: {len(fact_marketing)}")
print(f"Revenue weeks: {len(fact_revenue)}")


Data loaded:
Channels: 5
Marketing records: 172
Revenue weeks: 157


In [2]:
# Cell 2: Prepare MMM dataset (weekly spend per channel → wide format)
pivot_marketing = fact_marketing.pivot_table(
    index='date_key', 
    columns='channel_id', 
    values='spend', 
    aggfunc='sum'
).fillna(0).reset_index()

# Merge with revenue
mmm_data = pivot_marketing.merge(fact_revenue, on='date_key', how='inner')
print("MMM dataset shape:", mmm_data.shape)
print("\nFirst few rows:")
print(mmm_data.head())


MMM dataset shape: (101, 7)

First few rows:
     date_key    1      2    3    4      5   revenue
0  2021-01-25  0.0  0.032  0.0  0.0  0.000  46374.16
1  2021-02-01  0.0  0.032  0.0  0.0  0.141  53121.83
2  2021-02-08  0.0  0.032  0.0  0.0  0.141  49808.81
3  2021-02-15  0.0  0.055  0.0  0.0  0.141  50439.16
4  2021-02-22  0.0  0.023  0.0  0.0  0.141  54546.03


In [4]:
# Cell 3: Train the model
X = mmm_data.select_dtypes(include=[np.number]).drop(columns=['revenue'])  # All spend columns
y = mmm_data['revenue']

# Linear regression
model = Ridge(alpha=1.0)  # Ridge for stability
model.fit(X, y)

# Results
print("✅ Model trained!")
print("\nChannel contributions (spend → revenue impact):")
for i, channel in enumerate(dim_channel['channel_name']):
    coef = model.coef_[i]
    roi = coef * 100  # ROI proxy: 1.5 = $1 spend → $1.50 revenue
    print(f"{channel}: {coef:.3f} (ROI ~{roi:.0%})")

print(f"\nModel R²: {r2_score(y, model.predict(X)):.3f}")


✅ Model trained!

Channel contributions (spend → revenue impact):
Paid Search: 2161.708 (ROI ~21617082%)
Email: -1125.537 (ROI ~-11255374%)
Display: -2121.888 (ROI ~-21218877%)
Social: -1499.752 (ROI ~-14997515%)
Affiliate: 7702.652 (ROI ~77026523%)

Model R²: 0.039


In [6]:
# Cell 4: What-if simulator
def predict_revenue(spend_dict):
    """Input: {'Paid Search': 10000, 'Email': 5000, ...} → Output: predicted revenue"""
    spend_vector = []
    for cid in dim_channel['channel_id']:
        channel_name = dim_channel.loc[dim_channel['channel_id'] == cid, 'channel_name'].iloc[0]
        spend_vector.append(spend_dict.get(channel_name, 0))
    
    return model.predict(np.array(spend_vector).reshape(1, -1))[0]

# Test scenarios
baseline = {'Paid Search': 10000, 'Email': 5000, 'Display': 8000, 'Social': 12000, 'Affiliate': 3000}
optimized = {'Paid Search': 15000, 'Email': 3000, 'Display': 5000, 'Social': 15000, 'Affiliate': 5000}

print("Baseline spend →", f"${predict_revenue(baseline):,.0f} revenue")
print("Optimized spend →", f"${predict_revenue(optimized):,.0f} revenue")
print("Lift:", f"+${predict_revenue(optimized) - predict_revenue(baseline):,.0f}")

import joblib
joblib.dump(model, 'model.pkl')
joblib.dump(dim_channel, 'channels.pkl')
print("Model saved!")


Baseline spend → $4,178,448 revenue
Optimized spend → $34,509,777 revenue
Lift: +$30,331,329
Model saved!
